In [2]:
import syft as sy 
from torchvision import datasets
import torch

In [10]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 


Task exception was never retrieved
future: <Task finished name='Task-91' coro=<RTCSctpTransport._data_channel_flush() done, defined at C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py:1645> exception=ConnectionError('Cannot send encrypted data, not connected')>
Traceback (most recent call last):
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1676, in _data_channel_flush
    await self._send(
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1333, in _send
    await self._transmit()
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1556, in _transmit
    await self._send_chunk(chunk)
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1340, in _send_chun

♫♫♫ > DONE!

♫♫♫ > CONNECTED!


Exception in callback Transaction.__retry()
handle: <TimerHandle when=175432.546 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\21355\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...3\xe6_;`\x87'), ('192.168.1.5', 51574))>


In [6]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [13]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 448e8111d76d4a9d8b85d3ce4f396f53>,[DO1],This the MNIST dataset comming from the DO1 an...,<class 'torch.Tensor'>
1,<UID: 9e60210e833748589481ffb03600f220>,[DO1_test],This the MNIST dataset (test) comming from the...,<class 'torch.Tensor'>


In [14]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6332360e4add41e7a2804b4e78c98d5d>,[DO2],This the MNIST dataset comming from the DO2 an...,<class 'torch.Tensor'>
1,<UID: e1c5269e2d464a42bc46f6ab41eaa69e>,[DO2_test],This the MNIST dataset (test) comming from the...,<class 'torch.Tensor'>


In [19]:
labels = datasets.MNIST("mnist",download=True,train=True).targets[:25_000]
labels_val = datasets.MNIST("mnist",download=True,train=False).targets[:5000]

In [20]:
labels.shape

torch.Size([25000])

In [31]:
train1 = duet1.store[0]
val1 = duet1.store[1]

train2 = duet2.store[0]
val2 = duet2.store[1]

In [34]:
dl_1 = duet1.torch.utils.data.DataLoader(train1,batch_size = 128)
dl1_val = duet1.torch.utils.data.DataLoader(val1,batch_size = 128)



dl_2 = duet2.torch.utils.data.DataLoader(train2,batch_size = 128)
dl2_val = duet2.torch.utils.data.DataLoader(val2,batch_size = 128)

dl_local = torch.utils.data.DataLoader(labels,batch_size = 128)
dl1_val_local = torch.utils.data.DataLoader(labels_val,batch_size = 128)


In [36]:
# In order to serialise our model we need to define it as below
from torch import nn

hidden_sizes = [128, 500]
output_size = 10

class SyNet_client(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_client, self).__init__(torch_ref=torch_ref)
        self.lin = self.torch_ref.nn.Linear(392,64)
        
    def forward(self, x):
        x = self.lin(x)
        x = self.torch_ref.nn.functional.relu(x)
        return x
    
class SyNet_server(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_server, self).__init__(torch_ref=torch_ref)
        self.lin2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.lin3 = nn.Linear(hidden_sizes[1], output_size)
        self.sft = nn.LogSoftmax(dim=1) 
        
    def forward(self, x):
        x = self.lin2(x)
        x = nn.ReLU()(x)
        x = self.lin3(x)
        x = self.sft(x)
        return x

In [38]:
#Model 1
model1 = SyNet_client(torch)
model1_ptr = model1.send(duet1)
opt1 = duet1.torch.optim.SGD(params=model1_ptr.parameters(),lr=0.01)

#Model 2
model2 = SyNet_client(torch)
model2_ptr = model2.send(duet2)
opt2 = duet2.torch.optim.SGD(params=model2_ptr.parameters(),lr=0.01)

#Model 3
model3 = SyNet_server(torch)
local_opt = torch.optim.SGD(params=model3.parameters(),lr=0.1)

In [44]:
for epoch in range(4):
    train_correct = 0
    train_total = 0
    
    for in1, in2, label in zip(dl_1, dl_2, dl_local):
        opt1.zero_grad()
        opt2.zero_grad()
        local_opt.zero_grad()

        a1_ptr = model1_ptr(in1)
        a1 = a1_ptr.clone().get(request_block=True)

        a2_ptr = model2_ptr(in2)
        a2 = a2_ptr.clone().get(request_block=True)
        
        a = torch.cat((a1, a2), 1)
        y = model3(a)
        
        
        criterion = nn.NLLLoss()
        loss = criterion(y, label.long())
        print(y.max(1))
        train_correct += y.max(1)[1].eq(label).sum().item()
        train_total += y.shape[0]
        print(loss)
        
        loss.backward()

        g1_ptr = a1.grad.clone().send(duet1)
        g2_ptr = a2.grad.clone().send(duet2)
        
        a1_ptr.backward(g1_ptr)
        a2_ptr.backward(g2_ptr)
        
        local_opt.step()
        opt1.step()
        opt2.step()
        
    correct = 0
    total = 0
    for in_val1, in_val2, label_val in zip(dl_1val, dl_2val, dl_localval):
        a1_ptr = model1_ptr(in_val1)
        a1 = a1_ptr.clone().get(request_block=True)

        a2_ptr = model2_ptr(in_val2)
        a2 = a2_ptr.clone().get(request_block=True)

        a = torch.cat((a1, a2), 1)
        y = model3(a)

        correct += y.max(1)[1].eq(label_val).sum().item()
        total += y.shape[0]
    print(f"Train Accuracy: {100*train_correct/train_total:.3f}%")
    print(f"Val Accuracy: {100*correct/total:.3f}%")
        

torch.return_types.max(
values=tensor([-0.4527, -0.0163, -0.6196, -0.0474, -0.1980, -0.2295, -0.0449, -0.0596,
        -0.0317, -0.0734, -0.0163, -0.7956, -0.0116, -0.0907, -0.0290, -0.1606,
        -0.0271, -0.1588, -0.6409, -0.8362, -0.1063, -0.0232, -0.7405, -0.0505,
        -1.4806, -0.4328, -0.7322, -0.0059, -0.5703, -0.6529, -0.5769, -0.4653,
        -0.0301, -0.1518, -0.2966, -0.9607, -0.0333, -0.0145, -0.4565, -0.0362,
        -0.0159, -0.1980, -0.0681, -0.5092, -0.2630, -0.1591, -0.3414, -0.0445,
        -0.5326, -0.0894, -0.1109, -0.0266, -0.0125, -1.0462, -0.5441, -0.1778,
        -0.0275, -0.3368, -0.3869, -0.0620, -0.6880, -0.6127, -0.0832, -0.0465,
        -1.3801, -0.9129, -0.0147, -0.0150, -0.8006, -0.1135, -0.5652, -0.0788,
        -0.0217, -0.0516, -0.0672, -0.0599, -0.3798, -0.0250, -0.2113, -0.0931,
        -0.8149, -0.1366, -0.0067, -0.0873, -0.0300, -0.4572, -0.6545, -0.2022,
        -0.0522, -0.1090, -0.0040, -0.0628, -0.1644, -0.0547, -0.0716, -0.0212,
        -

KeyboardInterrupt: 